In [1]:
import json
import numpy as np
from torchvision import datasets, transforms
import redis
import time

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, weights, bias, activation, is_final_layer=False, dropout_rate=0.0):
        self.weights = np.array(weights) * (1 - dropout_rate)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Compute neuron output"""
        z = np.dot(inputs, self.weights) + self.bias  # (num_features,)
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        """Process one input sample at a time"""
        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)
        return outputs

# Build network
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for node in layer_info['nodes']
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

# Forward pass for single image
def forward_pass(layers, input_data):
    for layer in layers:
        input_data = layer.forward(input_data)
    return np.argmax(input_data)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
correct_predictions = 0
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    prediction = forward_pass(network, image_np)
    print(f"Image {i} Prediction: {prediction}, Label: {label}")
    if prediction == label:
        correct_predictions += 1

accuracy = correct_predictions / 10
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Image 0 Prediction: 7, Label: 7
Image 1 Prediction: 2, Label: 2
Image 2 Prediction: 1, Label: 1
Image 3 Prediction: 0, Label: 0
Image 4 Prediction: 4, Label: 4
Image 5 Prediction: 1, Label: 1
Image 6 Prediction: 4, Label: 4
Image 7 Prediction: 9, Label: 9
Image 8 Prediction: 6, Label: 5
Image 9 Prediction: 9, Label: 9
Test Accuracy: 90.00%
